In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings(action='ignore')
from math import atan, sqrt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import joblib

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/data/2021_04_19_검증set/Train_중력수정.csv')
train_labels=pd.read_csv('/content/drive/MyDrive/data/2021_04_19_검증set/Train_label(점수).csv')

In [ ]:
train = train.iloc[:,1:]

In [ ]:
#acc_x,y,z 칼럼만 가져오기
train_acc = train.iloc[:, 2:5]

In [ ]:
# column 추가 
# SVM (Sum Vector Magnitude)
# Roll (Acc Sensor X axis Angle value)
# Pitch (Acc Sensor Y axis Angle value)

train['SVM'] = (train['X']**2 + train['Y']**2 + train['Z']**2)**(1/2)
train['roll'] = (train.iloc[:,3]/(train.iloc[:,2]**2 + train.iloc[:,4]**2).apply(lambda x : sqrt(x))).apply(lambda x : atan(x))*180/np.pi
train['pitch'] = (train.iloc[:,2]/(train.iloc[:,3]**2 + train.iloc[:,4]**2).apply(lambda x : sqrt(x))).apply(lambda x : atan(x))*180/np.pi

In [ ]:
# 1Hz 기준이므로 1초로 미분
dt=1
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [ ]:
train_dt=[]

for i in train['id'].unique():
    temp=train.loc[train['id']==i]
    for v in train.columns[2:5]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        # 0초의 변화량은 0임으로 0으로 설정 
        temp.loc[:,v+'_diff']=values
    train_dt.append(temp)
    
train_diff=pd.concat(train_dt)

# 컬럼 재정렬
train_diff = train_diff[['id','X','Y','Z','SVM','roll','pitch','X_diff','Y_diff','Z_diff']]

In [ ]:
col=train_diff.columns
train_s=train_diff.copy()
col=train_s.columns

In [ ]:
scaler = joblib.load('/content/drive/MyDrive/Model/scaler2.pkl')

In [ ]:
train_s.iloc[:,1:]= scaler.transform(train_s.iloc[:,1:])
train_sc = pd.DataFrame(data = train_s,columns =col)